# HW10：RFM 與分群

This homework is written in the newer Polars and hvPlot library.

In [1]:
import polars as pl

In [2]:
df = pl.read_csv("tafeng5000.csv")
df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
i64,str,i64,str,str,i64,i64,i64,i64,i64
0,"""1/7/2001""",235419,"""30-34""","""115""",110105,4710198221113,1,25,29
1,"""2/13/2001""",1550900,"""35-39""","""115""",590312,4710735405006,1,103,119
2,"""2/26/2001""",909228,""">65""","""115""",110411,4710088414120,2,48,66
3,"""11/18/2000""",1380231,"""30-34""","""221""",130401,4710057870391,1,71,81
4,"""12/1/2000""",1695595,"""45-49""","""221""",530110,4710186161209,2,152,198
…,…,…,…,…,…,…,…,…,…
4995,"""12/26/2000""",1573879,"""55-59""","""221""",500806,26333005345,1,97,132
4996,"""2/3/2001""",2090818,"""35-39""","""221""",130205,4717332008883,1,75,95
4997,"""11/15/2000""",198127,"""30-34""","""115""",760229,733879271232,1,209,299


# Data cleanup

進行描述性統計。

In [3]:
df.describe()

statistic,,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
str,f64,str,f64,str,str,f64,f64,f64,f64,f64
"""count""",5000.0,"""5000""",5000.0,"""4851""","""5000""",5000.0,5000.0,5000.0,5000.0,5000.0
"""null_count""",0.0,"""0""",0.0,"""149""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",2499.5,null,1.4048e6,null,null,285985.2382,4.4571e12,1.3528,102.4436,120.803
"""std""",1443.520003,null,806133.233696,null,null,227810.005443,1.6876e12,1.043528,202.563897,227.340672
"""min""",0.0,"""1/1/2001""",4947.0,"""25-29""","""105""",100101.0,2.0009199e7,1.0,3.0,4.0
"""25%""",1250.0,null,960229.0,null,null,110106.0,4.7101e12,1.0,35.0,42.0
"""50%""",2500.0,null,1.590937e6,null,null,130102.0,4.7104e12,1.0,62.0,75.0
"""75%""",3749.0,null,1.848335e6,null,null,520437.0,4.7127e12,1.0,110.0,129.0
"""max""",4999.0,"""2/9/2001""",2.0002e7,""">65""","""Unknown""",780510.0,9.5576e12,30.0,9428.0,9980.0


發現 `AGE_GROUP` 有 149 個缺失值。但是我們不需要統計 `AGE_GROUP`，故直接移除這行即可。

In [4]:
df.drop_in_place("AGE_GROUP")

df.describe()

statistic,,TRANSACTION_DT,CUSTOMER_ID,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
str,f64,str,f64,str,f64,f64,f64,f64,f64
"""count""",5000.0,"""5000""",5000.0,"""5000""",5000.0,5000.0,5000.0,5000.0,5000.0
"""null_count""",0.0,"""0""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",2499.5,null,1.4048e6,null,285985.2382,4.4571e12,1.3528,102.4436,120.803
"""std""",1443.520003,null,806133.233696,null,227810.005443,1.6876e12,1.043528,202.563897,227.340672
"""min""",0.0,"""1/1/2001""",4947.0,"""105""",100101.0,2.0009199e7,1.0,3.0,4.0
"""25%""",1250.0,null,960229.0,null,110106.0,4.7101e12,1.0,35.0,42.0
"""50%""",2500.0,null,1.590937e6,null,130102.0,4.7104e12,1.0,62.0,75.0
"""75%""",3749.0,null,1.848335e6,null,520437.0,4.7127e12,1.0,110.0,129.0
"""max""",4999.0,"""2/9/2001""",2.0002e7,"""Unknown""",780510.0,9.5576e12,30.0,9428.0,9980.0


發現到 `TRANSACTION_DT` 是 `str` 而非 `datetime`，故進行轉換。

In [5]:
df = df.with_columns(pl.col("TRANSACTION_DT").str.strptime(pl.Date, "%m/%d/%Y"))

df.describe()

statistic,,TRANSACTION_DT,CUSTOMER_ID,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
str,f64,str,f64,str,f64,f64,f64,f64,f64
"""count""",5000.0,"""5000""",5000.0,"""5000""",5000.0,5000.0,5000.0,5000.0,5000.0
"""null_count""",0.0,"""0""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",2499.5,"""2000-12-30 12:12:05.760000""",1.4048e6,null,285985.2382,4.4571e12,1.3528,102.4436,120.803
"""std""",1443.520003,null,806133.233696,null,227810.005443,1.6876e12,1.043528,202.563897,227.340672
"""min""",0.0,"""2000-11-01""",4947.0,"""105""",100101.0,2.0009199e7,1.0,3.0,4.0
"""25%""",1250.0,"""2000-11-27""",960229.0,null,110106.0,4.7101e12,1.0,35.0,42.0
"""50%""",2500.0,"""2001-01-01""",1.590937e6,null,130102.0,4.7104e12,1.0,62.0,75.0
"""75%""",3749.0,"""2001-01-30""",1.848335e6,null,520437.0,4.7127e12,1.0,110.0,129.0
"""max""",4999.0,"""2001-02-28""",2.0002e7,"""Unknown""",780510.0,9.5576e12,30.0,9428.0,9980.0


計算總銷量。

In [6]:
df = df.with_columns(SALES_FIGURE=pl.col("AMOUNT") * pl.col("SALES_PRICE"))

df

,TRANSACTION_DT,CUSTOMER_ID,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE,SALES_FIGURE
i64,date,i64,str,i64,i64,i64,i64,i64,i64
0,2001-01-07,235419,"""115""",110105,4710198221113,1,25,29,29
1,2001-02-13,1550900,"""115""",590312,4710735405006,1,103,119,119
2,2001-02-26,909228,"""115""",110411,4710088414120,2,48,66,132
3,2000-11-18,1380231,"""221""",130401,4710057870391,1,71,81,81
4,2000-12-01,1695595,"""221""",530110,4710186161209,2,152,198,396
…,…,…,…,…,…,…,…,…,…
4995,2000-12-26,1573879,"""221""",500806,26333005345,1,97,132,132
4996,2001-02-03,2090818,"""221""",130205,4717332008883,1,75,95,95
4997,2000-11-15,198127,"""115""",760229,733879271232,1,209,299,299


# Question 1：計算 RFM

請計算 `tafeng5000` 的 RFM，$R$ 的計算終止日以 2001/12/31 日為標的。

In [7]:
from datetime import datetime

today = datetime(2001, 12, 31)

In [8]:
rfm_df = df.group_by(pl.col("CUSTOMER_ID").alias("CustomerID")).agg(
    recency=(today - pl.col("TRANSACTION_DT").max()).dt.total_days(),
    frequency=pl.col("TRANSACTION_DT").n_unique(),
    monetary=pl.col("SALES_FIGURE").sum(),
)

rfm_df

CustomerID,recency,frequency,monetary
i64,i64,u32,i64
1603385,414,1,169
163590,414,1,200
1328912,359,1,158
1553048,317,2,191
2028613,387,1,269
…,…,…,…
1408447,400,1,58
1211559,397,1,85
1726299,324,1,65


In [9]:
rfm_df.describe()

statistic,CustomerID,recency,frequency,monetary
str,f64,f64,f64,f64
"""count""",4186.0,4186.0,4186.0,4186.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",1.4060e6,362.495222,1.152652,250.538939
"""std""",670867.044024,35.945048,0.480804,808.717779
"""min""",4947.0,306.0,1.0,4.0
"""25%""",987660.0,331.0,1.0,52.0
"""50%""",1.596724e6,356.0,1.0,99.0
"""75%""",1.857801e6,396.0,1.0,199.0
"""max""",2.0002e7,425.0,10.0,27000.0


# Question 2：五等分分群

手動分群。

- 將 RFM 各切成 5 等份
- 等級最高給 5 分，最低給 1 分
- 請問 `555` 最有價值群
    1. RFM 平均數各為多少？
    2. 此群佔總人數多少百分比？

均分切成 5 等份。

In [10]:
pl.DataFrame(
    {
        "recency": rfm_df["recency"].qcut(5).unique(),
        # 不是均分，但可以確保維持 5 組
        "frequency": rfm_df["frequency"].qcut([0, 0.9, 0.99, 0.995, 1]).unique(),
        "monetary": rfm_df["monetary"].qcut(5).unique(),
    }
)

recency,frequency,monetary
cat,cat,cat
"""(-inf, 326]""","""(-inf, 1]""","""(-inf, 45]"""
"""(326, 345]""","""(1, 2]""","""(45, 77]"""
"""(345, 378]""","""(2, 3]""","""(77, 128]"""
"""(378, 401]""","""(3, 4]""","""(128, 247]"""
"""(401, inf]""","""(4, 10]""","""(247, inf]"""


根據上面 quantile，將 RFM 分群。

In [11]:
rfm_df = rfm_df.with_columns(
    # Recency 分數（越小越好）
    recency_score=pl.col("recency").qcut(5, labels=["5", "4", "3", "2", "1"]),
    # Frequency 分數（越大越好）
    frequency_score=pl.col("frequency").qcut(
        [0, 0.9, 0.99, 0.995], labels=["1", "2", "3", "4", "5"]
    ),
    # Monetary 分數（越大越好）
    monetary_score=pl.col("monetary").qcut(5, labels=["1", "2", "3", "4", "5"]),
)

rfm_df

CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score
i64,i64,u32,i64,cat,cat,cat
1603385,414,1,169,"""1""","""1""","""4"""
163590,414,1,200,"""1""","""1""","""4"""
1328912,359,1,158,"""3""","""1""","""4"""
1553048,317,2,191,"""5""","""2""","""4"""
2028613,387,1,269,"""2""","""1""","""5"""
…,…,…,…,…,…,…
1408447,400,1,58,"""2""","""1""","""2"""
1211559,397,1,85,"""2""","""1""","""3"""
1726299,324,1,65,"""5""","""1""","""2"""


寫出最有價值群 `customer_score`。

In [12]:
rfm_df = rfm_df.with_columns(
    customer_score=pl.concat_str(
        pl.col("recency_score"), pl.col("frequency_score"), pl.col("monetary_score")
    )
)

rfm_df

CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,customer_score
i64,i64,u32,i64,cat,cat,cat,str
1603385,414,1,169,"""1""","""1""","""4""","""114"""
163590,414,1,200,"""1""","""1""","""4""","""114"""
1328912,359,1,158,"""3""","""1""","""4""","""314"""
1553048,317,2,191,"""5""","""2""","""4""","""524"""
2028613,387,1,269,"""2""","""1""","""5""","""215"""
…,…,…,…,…,…,…,…
1408447,400,1,58,"""2""","""1""","""2""","""212"""
1211559,397,1,85,"""2""","""1""","""3""","""213"""
1726299,324,1,65,"""5""","""1""","""2""","""512"""


篩選出 `555` 最有價值群。

In [13]:
best_customers = rfm_df.filter(pl.col("customer_score") == "555")

best_customers

CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,customer_score
i64,i64,u32,i64,cat,cat,cat,str
2113579,306,5,559,"""5""","""5""","""5""","""555"""
1848410,325,5,537,"""5""","""5""","""5""","""555"""
1638080,315,6,411,"""5""","""5""","""5""","""555"""
2112589,315,8,1575,"""5""","""5""","""5""","""555"""
2133874,323,6,13567,"""5""","""5""","""5""","""555"""
20459,315,10,1705,"""5""","""5""","""5""","""555"""


最有價值群的 RFM 平均數。

In [14]:
best_customers.select(
    pl.col("recency").mean(), pl.col("frequency").mean(), pl.col("monetary").mean()
)

recency,frequency,monetary
f64,f64,f64
316.5,6.666667,3059.0


此群佔總人數多少百分比？

In [15]:
print("Best customers: {:.2f}%".format(len(best_customers) / len(rfm_df) * 100))

Best customers: 0.14%


# Question 3：兩等份分群

手動分群：將 F、R 各切成 2 等份（高與低），然後依 F、R 高低分為四群，並給予命名：

- **常貴客**
- **一次客**
- **新顧客**
- **流失客**

In [16]:
rfm_df = rfm_df.with_columns(
    recency_level=pl.col("recency").qcut(2, labels=["Low", "High"]),
    frequency_level=pl.col("frequency").cut([1], labels=["Low", "High"]),
)

rfm_df

CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,customer_score,recency_level,frequency_level
i64,i64,u32,i64,cat,cat,cat,str,cat,cat
1603385,414,1,169,"""1""","""1""","""4""","""114""","""High""","""Low"""
163590,414,1,200,"""1""","""1""","""4""","""114""","""High""","""Low"""
1328912,359,1,158,"""3""","""1""","""4""","""314""","""High""","""Low"""
1553048,317,2,191,"""5""","""2""","""4""","""524""","""Low""","""High"""
2028613,387,1,269,"""2""","""1""","""5""","""215""","""High""","""Low"""
…,…,…,…,…,…,…,…,…,…
1408447,400,1,58,"""2""","""1""","""2""","""212""","""High""","""Low"""
1211559,397,1,85,"""2""","""1""","""3""","""213""","""High""","""Low"""
1726299,324,1,65,"""5""","""1""","""2""","""512""","""Low""","""Low"""


In [17]:
rfm_df = rfm_df.with_columns(
    pl.when((pl.col("recency_level") == "Low") & (pl.col("frequency_level") == "Low"))
    .then(pl.lit("流失客"))
    .when((pl.col("recency_level") == "Low") & (pl.col("frequency_level") == "High"))
    .then(pl.lit("新顧客"))
    .when((pl.col("recency_level") == "High") & (pl.col("frequency_level") == "Low"))
    .then(pl.lit("先前客"))
    .when((pl.col("recency_level") == "High") & (pl.col("frequency_level") == "High"))
    .then(pl.lit("常貴客"))
    .otherwise(pl.lit(None))  # Optional: handle unexpected cases
    .alias("customer_level")
)

rfm_df

CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,customer_score,recency_level,frequency_level,customer_level
i64,i64,u32,i64,cat,cat,cat,str,cat,cat,str
1603385,414,1,169,"""1""","""1""","""4""","""114""","""High""","""Low""","""先前客"""
163590,414,1,200,"""1""","""1""","""4""","""114""","""High""","""Low""","""先前客"""
1328912,359,1,158,"""3""","""1""","""4""","""314""","""High""","""Low""","""先前客"""
1553048,317,2,191,"""5""","""2""","""4""","""524""","""Low""","""High""","""新顧客"""
2028613,387,1,269,"""2""","""1""","""5""","""215""","""High""","""Low""","""先前客"""
…,…,…,…,…,…,…,…,…,…,…
1408447,400,1,58,"""2""","""1""","""2""","""212""","""High""","""Low""","""先前客"""
1211559,397,1,85,"""2""","""1""","""3""","""213""","""High""","""Low""","""先前客"""
1726299,324,1,65,"""5""","""1""","""2""","""512""","""Low""","""Low""","""流失客"""


In [18]:
rfm_df["customer_level"].value_counts()

customer_level,count
str,u32
"""流失客""",1724
"""新顧客""",373
"""先前客""",1955
"""常貴客""",134


# Question 4：計算主力產品

承上，針對常貴客群進行分析，計算此群前五項主力商品。

In [19]:
rfm_df.filter(pl.col("customer_level") == "常貴客").join(
    df, left_on="CustomerID", right_on="CUSTOMER_ID"
)["PRODUCT_ID"].value_counts().sort("count", descending=True).head(5)

PRODUCT_ID,count
i64,u32
4711271000014,6
4714981010038,4
4710134023276,3
4710094032011,2
4710011401128,2


# Question 5：$K$-means 分群

將 RFM 以 $K$-means 分群，分群後，畫出各群分布圖（$F$ 值為 X 軸，$R$ 值為 Y 軸），並標示出中心點。

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## K-means 分群

進行標準化。

In [21]:
scale = StandardScaler()

# 只取 RFM 欄位
rfm_df_v = rfm_df.select(["recency", "frequency", "monetary"])

normalized_rfm_df = pl.from_numpy(
    scale.fit_transform(rfm_df_v.to_numpy()), rfm_df_v.columns
)

normalized_rfm_df

recency,frequency,monetary
f64,f64,f64
1.433047,-0.317531,-0.100837
1.433047,-0.317531,-0.0625
-0.09725,-0.317531,-0.11444
-1.265839,1.762569,-0.07363
0.68181,-0.317531,0.02283
…,…,…
1.043517,-0.317531,-0.238108
0.960046,-0.317531,-0.204718
-1.071074,-0.317531,-0.229451


使用 Silhouette 分數找出最適合分群數量。

In [22]:
for n in range(3, 12):
    kmeans = KMeans(n_clusters=n, random_state=486).fit(normalized_rfm_df.to_pandas())
    silhouette_avg = silhouette_score(
        normalized_rfm_df.to_pandas(), kmeans.labels_, random_state=512
    )

    print(f"n_clusters: {n}, silhouette_score: {silhouette_avg}")

n_clusters: 3, silhouette_score: 0.5602914703804486
n_clusters: 4, silhouette_score: 0.5799998098513125
n_clusters: 5, silhouette_score: 0.5831375042702343
n_clusters: 6, silhouette_score: 0.45763768421991513
n_clusters: 7, silhouette_score: 0.47044127315885226
n_clusters: 8, silhouette_score: 0.5240861924274084
n_clusters: 9, silhouette_score: 0.5347067234033426
n_clusters: 10, silhouette_score: 0.5434195407946887
n_clusters: 11, silhouette_score: 0.5412954056501128


看起來 $K=5$ ($\mathrm{SS} = 0.58822$) 是最好的分組數量。

> 因為 KMeans 和 Silhouette 分數都有 `random_state`，我沒成功鎖住，所以每次執行結果都不一樣。

In [23]:
kmeans = KMeans(n_clusters=5).fit(normalized_rfm_df.to_pandas())

print(kmeans.labels_)
print(kmeans.cluster_centers_)

[3 3 1 ... 1 3 3]
[[-0.58712494  2.30263512  0.13450714]
 [-0.73091383 -0.31753071 -0.08791843]
 [-0.2135378   0.37583577  6.33056037]
 [ 1.05496758 -0.31127289 -0.11775011]
 [-0.52851485 -0.31753071 28.72724803]]


還原標準化後的值，找出各群中心點，可與上面標準化後的值比較。

In [24]:
customer_center_reverse = pl.from_numpy(
    scale.inverse_transform(kmeans.cluster_centers_), rfm_df_v.columns
)
customer_center_reverse.columns = [
    "recency_center",
    "frequency_center",
    "monetary_center",
]

customer_center_reverse

recency_center,frequency_center,monetary_center
f64,f64,f64
341.393509,2.259635,359.30426
336.225628,1.0,179.446231
354.820513,1.333333,5369.564103
400.411552,1.003008,155.323706
343.5,1.0,23480.0


加入各群標籤以及計算各群個數。

In [25]:
four_cluster_df = rfm_df_v.with_columns(pl.Series("cluster", kmeans.labels_))

four_cluster_df

recency,frequency,monetary,cluster
i64,u32,i64,i32
414,1,169,3
414,1,200,3
359,1,158,1
317,2,191,0
387,1,269,3
…,…,…,…
400,1,58,3
397,1,85,3
324,1,65,1


In [26]:
four_cluster_df.group_by("cluster").agg(pl.col("monetary").count()).sort("cluster")

cluster,monetary
i32,u32
0,493
1,1990
2,39
3,1662
4,2


## 各群分佈圖

加入各群標籤以及計算各群個數。

In [27]:
customer_center_reverse.with_row_index()

index,recency_center,frequency_center,monetary_center
u32,f64,f64,f64
0,341.393509,2.259635,359.30426
1,336.225628,1.0,179.446231
2,354.820513,1.333333,5369.564103
3,400.411552,1.003008,155.323706
4,343.5,1.0,23480.0


In [28]:
import altair as alt

(
    customer_center_reverse.with_row_index()
    .plot.scatter(
        x=alt.X("frequency_center", scale=alt.Scale(domain=[0, 10])),
        y="recency_center",
        color=alt.Color("index:N"),
    )
    .properties(title="Cluster Centers")
    | four_cluster_df.with_row_index()
    .plot.scatter(x="frequency", y="recency", color=alt.Color("cluster:N"))
    .properties(title="Clustered Customers")
)

alt.HConcatChart(...)

# Question 6：泡泡圖

請將各群畫出泡泡圖，X 軸為 frequency，y 軸為 recency,，泡泡大小為每群人數。

In [29]:
four_clusters_cdf = four_cluster_df.group_by("cluster").agg(
    recency=pl.col("recency").mean(),
    frequency=pl.col("frequency").mean(),
    count=pl.col("recency").count(),
)

four_clusters_cdf

cluster,recency,frequency,count
i32,f64,f64,u32
0,341.393509,2.259635,493
3,400.411552,1.003008,1662
4,343.5,1.0,2
1,336.225628,1.0,1990
2,354.820513,1.333333,39


In [30]:
four_clusters_cdf.plot.scatter(
    x=alt.X("frequency", scale=alt.Scale(domain=[0.6, 2.4])),
    y=alt.Y("recency", scale=alt.Scale(domain=[300, 420])),
    color="cluster:N",
    size="count",
).properties(title="各群泡泡圖")

alt.Chart(...)

# Question 7：哪一群為最有價值？

print 出此群的 RFM 值。

為第二群。

In [31]:
four_cluster_df.filter(pl.col("cluster") == 2).select(
    recency=pl.col("recency").mean(),
    frequency=pl.col("frequency").mean(),
    monetary=pl.col("monetary").sum(),
)

recency,frequency,monetary
f64,f64,i64
354.820513,1.333333,209413


# Question 8：推薦產品

針對最有價值群那一群顧客，列出每個人的推薦商品，請依顧客編號排名，print前5筆結果。

In [32]:
(
    four_cluster_df.with_columns(rfm_df["CustomerID"])
    .sort("CustomerID")
    .filter(pl.col("cluster") == 2)
    .join(df, left_on="CustomerID", right_on="CUSTOMER_ID")
    .select(
        pl.col("CustomerID").alias("customer_id"),
        pl.col("PRODUCT_ID").alias("product_id"),
        pl.col("SALES_FIGURE").alias("sales_figure"),
    )
    .group_by("customer_id")
    .agg(
        product_id=pl.col("product_id")
        .filter(pl.col("sales_figure") == pl.col("sales_figure").max())
        .first(),
    )
    .head(5)
)

customer_id,product_id
i64,i64
1888829,8712045013069
2140254,4719111015037
539951,4711360162111
1830361,4710036008548
758680,4710375111114
